In [22]:
import rasterio
from rasterio.features import shapes
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
from fiona.crs import from_epsg
from matplotlib.colors import LightSource
import geopandas as gpd
import requests
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import fiona
import pycrs
import json
import pprint
import plotly.graph_objs as go
import plotly.express as px
from datetime import datetime
from osgeo import gdal
from osgeo import ogr

In [23]:

DATAS =  {
    # 'LGE' : {'MNT': '/media/bestycame/407C5CD57C5CC776/3D/RELIEF_WALLONIE_MNT_2013_2014_LIEGE.tif',
    #                'MNS': '/media/bestycame/407C5CD57C5CC776/3D/RELIEF_WALLONIE_MNS_2013_2014_LIEGE.tif'},
          'HAI' : {'MNT': '/Volumes/T7/Data/RELIEF_WALLONIE_MNT_2013_2014_GEOTIFF_31370_PROV_HAINAUT/RELIEF_WALLONIE_MNT_2013_2014.tif',
                   'MNS': '/Volumes/T7/Data/RELIEF_WALLONIE_MNS_2013_2014_GEOTIFF_31370_PROV_HAINAUT/RELIEF_WALLONIE_MNS_2013_2014.tif'},
          # 'LUX' : {'MNT': '/mnt/3D/RELIEF_WALLONIE_MNT_2013_2014_LUXEMBOURG.tif',
          #          'MNS': '/mnt/3D/RELIEF_WALLONIE_MNS_2013_2014_LUXEMBOURG.tif'},
          # 'NAM' : {'MNT': '/mnt/3D/RELIEF_WALLONIE_MNT_2013_2014_NAMUR.tif',
          #          'MNS': '/mnt/3D/RELIEF_WALLONIE_MNS_2013_2014_NAMUR.tif'},
          # 'BRA' : {'MNT': '/mnt/3D/RELIEF_WALLONIE_MNT_2013_2014_BRABANT.tif',
          #          'MNS': '/mnt/3D/RELIEF_WALLONIE_MNS_2013_2014_BRABANT.tif'}
                   }
province = "HAINAUT"
MNS_MNT = "MNS"
personal_path = "/Volumes/T7/Data"
file_path = f"{personal_path}/RELIEF_WALLONIE_{MNS_MNT}_2013_2014_{province}.tif"

In [24]:
# gdal_calc.py -A DSM/GeoTIFF/DHMVII_DSM_1m_testdata.tif -B ./dtm_1m.tif --outfile=CHM.tif --calc="A-B" --NoDataValue=0
# gdalwarp  DTM/GeoTIFF/DHMVII_DTM_5m_testdata.tif test.tif -tr 1 1 -r near
# gdal_contour -3d -a elev chm.tif contour.shp -i 0.5 -snodata 0b

In [25]:
# gdb = fiona.open('./BATI3D_2013-2014_FILEGDB_31370/BATI3D_2013-2014.gdb', layer=1)
# hits = list(gdb.items(bbox=(xMin, yMin, xMax, yMax)))
# meshes = []
# for hit in hits:
#     print(hit)
    # points = hit[1]['geometry']['coordinates']
    # pts = [item for sublist in points for sitem in sublist for item in sitem]
    # pcd = o3d.geometry.PointCloud()
    # pcd.points = o3d.utility.Vector3dVector(pts)
    # mesh = pcd.compute_convex_hull()
    # meshes.append(mesh[0])
# o3d.visualization.draw_geometries(meshes, point_show_normal=True, mesh_show_wireframe=True, mesh_show_back_face=True)

In [26]:
# Christophe Schenke 
# christophe.shanke@spw.walonnie.be  
# 0475 95 4570

In [27]:
from pyproj import Transformer
def lambert_to_wgs(x_lambert, y_lambert):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326")
    x_wgs, y_wgs = transformer.transform(x_lambert, y_lambert)
    return x_wgs, y_wgs

def wgs_to_lambert(x_wgs, y_wgs):
    transformer = Transformer.from_crs("epsg:4326", "epsg:31370")
    x_lambert, y_lambert = transformer.transform(x_wgs, y_wgs)
    return x_lambert, y_lambert

# x, y, xMin, xMax, yMin, yMax = search_address(cp='5000', rue='RUE DE LA CRETE', num = '111', as_wgs=False)
def search_address(cp, rue, num, as_wgs=False, as_dict=False, boundary=False):    
    url = 'http://geoservices.wallonie.be/geolocalisation/rest/getPositionByCpRueAndNumero/{cp}/{rue}/{num}'
    r = requests.get(url.format(cp=cp, rue=rue, num=num)).json()
    x, y, xMin, xMax, yMin, yMax = r['x'], r['y'], r['rue']['xMin'], r['rue']['xMax'], r['rue']['yMin'], r['rue']['yMax']
    if as_wgs:
        x, y       = lambert_to_wgs(x, y)
        xMin, xMax = lambert_to_wgs(xMin, xMax)
        yMin, yMax = lambert_to_wgs(yMin, yMax)
    if as_dict:
        return {'x': x, 'xMin': xMin, 'xMax': xMax, 
                'y': y, 'yMin': yMin, 'yMax': yMax}
    return x, y, xMin, xMax, yMin, yMax

#x, y, xMin, xMax, yMin, yMax = search_address_mapbox("Rue du roton 38, 6000 Charleroi", as_wgs=False, boundary=150)
def search_address_mapbox(address, as_wgs=False, as_dict=False, boundary=200):
    key = 'pk.eyJ1IjoiYmFjYXlhdzU1OSIsImEiOiJja2dtNTgyNW8wMWN2MnBzM2loNGt4NmlzIn0.3IYulU7qzfpq2Ms0aaYWMQ'
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{address}.json?types=address&access_token={key}"
    r = requests.get(url.format(address=address,key=key))
    if r.status_code != 200:
        return 'NotFound'
    r = r.json()
    y, x = r['features'][0]['center']
    x, y = wgs_to_lambert(x, y)
    xMin, xMax = x - boundary, x + boundary
    yMin, yMax = y - boundary, y + boundary
    if as_wgs:
        x, y       = lambert_to_wgs(x, y)
        xMin, xMax = lambert_to_wgs(xMin, xMax)
        yMin, yMax = lambert_to_wgs(yMin, yMax) 
    if as_dict:
        return {'x': x, 'xMin': xMin, 'xMax': xMax, 
                'y': y, 'yMin': yMin, 'yMax': yMax}
    return x, y, xMin, xMax, yMin, yMax

def is_in_bbox(x, y, bbox):
    xMin, yMin, xMax, yMax = bbox
    if xMin <= x <= xMax and  yMin <= y <= yMax:
        return True
    return False

def find_files(xMin, xMax, yMin, yMax):
    bboxes = {key:rasterio.open(next(iter(DATAS[key].values()))).bounds for key in DATAS.keys()}
    for k, bbox in bboxes.items():
        if is_in_bbox(xMin, yMin, bbox) and is_in_bbox(xMax, yMax, bbox):
            return DATAS[k]
    return 'File not found'

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

def subsetTif(xMin, xMax, yMin, yMax, in_tif, out_tif=None):
    if out_tif == None:
        out_tif = datetime.now().strftime('%Y%m%d_%H%M%S.%f') + '.tif'
    data = rasterio.open(in_tif)
    geo = gpd.GeoDataFrame({'geometry': box(xMin, yMin, xMax, yMax)}, index=[0], crs=from_epsg(31370))
    out_img, out_transform = mask(dataset=data, shapes=getFeatures(geo), filled=False, crop=True)
    out_meta = data.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_img.shape[1],
                 "width": out_img.shape[2],
                 "transform": out_transform,
                "crs": pycrs.parse.from_epsg_code(31370).to_proj4()
                    })
    with rasterio.open(out_tif, "w", **out_meta) as dest:
        dest.write(out_img)
    return out_tif

#show_tif('subset.tif', title=address, hillshade=False)
def show_tif(tif, title=None, hillshade=False):
    if title == None:
        title = tif
    colors = [[0.0, 'rgb(000, 063, 076)'], [0.1, 'rgb(029, 081, 059)'], [0.2, 'rgb(055, 098, 043)'], 
             [0.3, 'rgb(079, 114, 030)'], [0.4, 'rgb(103, 129, 016)'], [0.5, 'rgb(136, 142, 002)'], 
             [0.6, 'rgb(169, 154, 021)'], [0.7, 'rgb(192, 171, 045)'], [0.8, 'rgb(214, 188, 074)'],
             [0.9, 'rgb(234, 209, 112)'], [1.0, 'rgb(254, 229, 152)']]
    
    with rasterio.open(tif) as src:
        data = src.read(1)

    if hillshade:
        light = LightSource(azdeg=315, altdeg=45)
        data = light.hillshade(data, vert_exag=1)     

    fig = px.imshow(data, color_continuous_scale=colors)
    fig.update_traces(customdata=data, hovertemplate='elevation: %{customdata:.4f}')
    fig.update_layout(**dict(title_text=title, width=700, height=500, template='none'))
    return fig

def create_chm(xMin, xMax, yMin, yMax):
    files = find_files(xMin, xMax, yMin, yMax)
    time = datetime.now().strftime('%Y%m%d_%H%M%S.%f') + '.tif'
    for k, file in files.items():
        subsetTif(xMin, xMax, yMin, yMax, file, k +'_'+time)
    !gdal_calc.py -A MNS_$time -B MNT_$time --outfile=CHM_$time --calc="A-B" --NoDataValue -9999 
    return time

In [57]:
# print(DATAS['HAI']["MNS"])
address = "rue du roton 38, 6000 Charleroi"
x, y, xMin, xMax, yMin, yMax = search_address_mapbox(address, as_wgs=False, boundary=50)
time = create_chm(xMin, xMax, yMin, yMax)
print(lambert_to_wgs(xMin,yMin))
print(lambert_to_wgs(xMax,yMax))

/Users/samueldodet/Documents/current project/6.3D_houses/venv/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



zsh:1: /Users/samueldodet/Documents/current project/6.3D_houses/venv/bin/gdal_calc.py: bad interpreter: /Users/samueldodet/Documents/current: no such file or directory
ERROR 1: PROJ: proj_create_from_database: Cannot find proj.db
ERROR 1: PROJ: proj_create_from_database: Cannot find proj.db
ERROR 1: PROJ: proj_create_from_name: Cannot find proj.db
0.. 16.. 33.. 50.. 66.. 83.. Dataset writing failed
(50.42083297121772, 4.44776464147411)
(50.42173102980425, 4.449173363541769)


In [29]:
print(fiona.supported_drivers)
print('test2')
driver = ogr.GetDriverByName("FileGDB")


gdb = fiona.open('/Users/samueldodet/Documents/current project/6.3D_houses/BATI3D_2013-2014_FILEGDB_31370/BATI3D_2013-2014.gdb', layer=0)
hits = list(gdb.items(bbox=(xMin, yMin, xMax, yMax)))

features = []
for hit in hits:
    features.append({'id':           hit[0],
                        'H_MUR':        hit[1]['properties']['H_MUR'],
                        'H_TOIT':       hit[1]['properties']['H_TOIT'],
                        'E_TOIT':       hit[1]['properties']['E_TOIT'],
                        'Q_LIDAR':      hit[1]['properties']['Q_LIDAR'],
                        'Q_BATI' :      hit[1]['properties']['Q_BATI'],
                        'SHAPE_Length': hit[1]['properties']['SHAPE_Length'],
                        'SHAPE_Area':   hit[1]['properties']['SHAPE_Area'],
                        'X': [int(x[0]) for x in hit[1]['geometry']['coordinates'][0][0]],
                        'Y': [int(x[1]) for x in hit[1]['geometry']['coordinates'][0][0]]
                       })

{'ARCGEN': 'r', 'DXF': 'rw', 'CSV': 'raw', 'OpenFileGDB': 'r', 'ESRIJSON': 'r', 'ESRI Shapefile': 'raw', 'FlatGeobuf': 'rw', 'GeoJSON': 'raw', 'GeoJSONSeq': 'rw', 'GPKG': 'raw', 'GML': 'rw', 'OGR_GMT': 'rw', 'GPX': 'rw', 'GPSTrackMaker': 'rw', 'Idrisi': 'r', 'MapInfo File': 'raw', 'DGN': 'raw', 'OGR_PDS': 'r', 'S57': 'r', 'SQLite': 'raw', 'TopoJSON': 'r'}
test2


In [64]:
data = rasterio.open(f'CHM_{time}')
xMin, yMin, xMax, yMax = data.bounds
z_data = pd.DataFrame(data.read(1))
z_data.index   = list(range(int(yMax), int(yMin), -1))
z_data.columns = list(range(int(xMin), int(xMax)))
z_data = z_data.applymap(lambda x: 0 if x < 0.5 else round(x, 1))
print(z_data)

fig = go.Figure(go.Surface(z=z_data.values, x=z_data.columns, y=z_data.index, opacity=1, colorscale="greens"))

for feature in features:
    fig.add_scatter3d(x=feature['X'], y=feature['Y'], z=[3]*len(feature['X']),
                      mode='lines',showlegend=False,
                      line=dict(color='red', width=10)
                     )

fig.update_layout(
    title=address,
    width=900,
    height=1000,
    margin=dict(t=40, r=0, l=0, b=40),
    scene = {"xaxis": {'showspikes': False},
             "yaxis": {'showspikes': False},
             "zaxis": {'showspikes': False},
             'camera_eye': {"x": 0, "y": -0.5, "z": 0.5},
             "aspectratio": {"x": 1, "y": 1, "z": 0.1}
            })
fig.show()
print("tes2")

        155615  155616  155617  155618  155619  155620  155621  155622  \
123431     0.0     0.0     0.0     0.0     0.0     0.0     0.6     0.0   
123430     0.0     0.0     0.0     0.0     0.0     0.0     0.6     0.0   
123429     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
123428     8.5     8.4     0.0     0.0     0.0     0.0     0.0     0.0   
123427     8.5     8.4     8.3     0.0     8.2     0.0     0.0     0.0   
...        ...     ...     ...     ...     ...     ...     ...     ...   
123335     6.1     4.0     3.0     2.8     2.4     2.2     2.1     0.0   
123334     5.9     2.4     2.4     2.8     2.3     2.2     1.1     0.0   
123333     5.7     2.8     2.9     2.4     2.2     2.1     0.0     0.0   
123332     5.6     2.7     2.7     2.3     2.1     2.0     0.5     0.0   
123331     2.8     3.1     2.6     2.3     2.1     0.0     0.0     0.0   

        155623  155624  ...  155706  155707  155708  155709  155710  155711  \
123431     0.0     0.0  ...     

tes2


In [66]:
import polyline
data = rasterio.open(f'CHM_{time}')
xMin, yMin, xMax, yMax = data.bounds
z_data = pd.DataFrame(data.read(1))
z_data.index   = list(range(int(yMax), int(yMin), -1))
z_data.columns = list(range(int(xMin), int(xMax)))
z_data = z_data.applymap(lambda x: 0 if x < 5 else round(x, 1))
array_z =np.array(z_data)
print(array_z)
# polyline.encode(z_data)
fig = go.Figure(go.Surface(z=z_data.values, x=z_data.columns, y=z_data.index, opacity=1,colorscale="greens"))
# for feature in features:
#     fig.add_scatter3d(x=feature['X'], y=feature['Y'], z=[3]*len(feature['X']),
#                       mode='lines',showlegend=False,
#                       line=dict(color='red', width=10))
#
fig.update_layout(
    title=address,
    width=900,
    height=1000,
    margin=dict(t=40, r=0, l=0, b=40),
    scene = {
        "xaxis": {'showspikes': False},
             "yaxis": {'showspikes': False},
             "zaxis": {'showspikes': False},
             'camera_eye': {"x": 0, "y": -0.5, "z": 0.5},
             "aspectratio": {"x": 1, "y": 1, "z": 0.1}
            })


fig.show()

[[0.  0.  0.  ... 0.  0.  6.8]
 [0.  0.  0.  ... 6.1 7.3 6. ]
 [0.  0.  0.  ... 7.8 7.2 6.1]
 ...
 [5.7 0.  0.  ... 0.  0.  0. ]
 [5.6 0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]


In [54]:
import pandas as pd
gdb = fiona.open('./BATI3D_2013-2014_FILEGDB_31370/BATI3D_2013-2014.gdb', layer=1)
hits = list(gdb.items(bbox=(xMin, yMin, xMax, yMax)))
meshes = []
for hit in hits:
    points = hit[1]['geometry']['coordinates']
    meshes.append(points)
df= pd.DataFrame(meshes)
x_1= []
y_1= []
z_1= []
list_point=[]
for mesh in meshes:
    for i in mesh:
        for a in i:
            for b in a:
                list_point.append(b)
                x_1.append(round(b[0]))
                y_1.append(round(b[1]))
                z_1.append(round(b[2]))
# print(list_point)
test_list= []
# print(list_point)

for point in list_point:
    point = list(point)
    if round(point[0]) in z_data.columns and round(point[1]) in z_data.index:
        # print(z_data.at[round(point[1]),round(point[0])])
        point[2] = z_data.at[round(point[1]),round(point[0])]
        # test_list.append((point[2]))
        test_list.append(point)
# for el in test_list:
#     a=np.array(el)
#     s=np.isnan(a)
#     a[s]=0.0
print("fze")
array_test = np.array(test_list)


print(array_test)

fig = go.Figure(go.Surface(z=array_test[:,2], x=array_test[:,0], y=array_test[:,1], opacity=1))
fig.update_layout(
    title=address,
    width=900,
    height=1000,
    margin=dict(t=40, r=0, l=0, b=40),
    scene = {
        "xaxis": {'showspikes': False},
             "yaxis": {'showspikes': False},
             "zaxis": {'showspikes': False},
             'camera_eye': {"x": 0, "y": -0.5, "z": 0.5},
             "aspectratio": {"x": 1, "y": 1, "z": 0.1}
            })

fig.show()
print("lel")

fze
[['155618.375' '123332.75400000066' 'Nan']
 ['155615.186999999' '123333.64200000092' '5.9']
 ['155617.65200000256' '123330.68600000069' 'Nan']
 ...
 ['155619.50299999863' '123426.0' '8.1']
 ['155621.36900000274' '123425.01000000164' '7.9']
 ['155616.20000000298' '123415.49799999967' '8.9']]


In [ ]:
! pip install PyQt5

In [ ]:
! pip install gdal

In [ ]:
from osgeo import gdal
from mayavi import mlab

ds = gdal.Open('CHM_20210629_111234.988476.tif')
# data = ds.ReadAsArray()
# mlab.figure(size=(640, 800), bgcolor=(0.16, 0.28, 0.46))
# mlab.surf(data, warp_scale=0.2)
# mlab.savefig('test.png')
# mlab.show()

